In [82]:
import pandas as pd
import numpy as np

import yfinance as yf


In [83]:
df = pd.read_excel(r'C:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\ESG_DATA1.xlsx')

def ESG_SCORE_WEIGHT(data, weights):
    data['weighted_score'] = np.sum([data[col] * weight for col, weight in zip(['environment_score', 'governance_score', 'social_score'], weights)])
    return data

weights = np.array([0.4, 0.3, 0.3])

# Group the DataFrame by 'Isin' and apply the ESG_SCORE_WEIGHT function to each group
weighted_scores = df.groupby('Isin').apply(ESG_SCORE_WEIGHT, weights=weights)

for index, row in weighted_scores.iterrows():
    if row['CurrencyCode'] == 'SEK':
        weighted_scores.at[index, 'stock_symbol'] += '.ST'
    elif row['CurrencyCode'] == 'NOK':
        weighted_scores.at[index, 'stock_symbol'] += '.OL'
    elif row['CurrencyCode'] == 'DKK':
        weighted_scores.at[index, 'stock_symbol'] += '.CO'

    



weighted_scores

,Isin,CurrencyCode,company_name,environment_grade,environment_level,environment_score,governance_grade,governance_level,governance_score,social_grade,social_level,social_score,stock_symbol,total,total_grade,total_level,weighted_score
0,SE0011337708,SEK,AAK AB (publ.),AA,Excellent,647,BB,Medium,345,BB,Medium,307,AAK.ST,1299,A,High,454.4
1,CH0012221716,SEK,ABB Ltd,BB,Medium,306,B,Medium,220,BBB,High,425,ABB.ST,951,BBB,High,315.9
2,SE0007897079,SEK,ACADIA Pharmaceuticals Inc.,B,Medium,290,BB,Medium,300,BB,Medium,344,ACAD.ST,934,BBB,High,309.2
3,SE0014781795,SEK,Addtech AB (publ.),A,High,500,BB,Medium,300,BB,Medium,300,ADDT-B.ST,1100,BBB,High,380.0
4,CA00830W1059,SEK,AEC Securities Public Company Limited,B,Medium,240,BB,Medium,300,BB,Medium,300,AEC.ST,840,BB,Medium,276.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,DK0060036564,DKK,Spar Nord Bank A/S,B,Medium,230,BB,Medium,300,B,Medium,216,SPNO.CO,746,B,Medium,246.8
283,DK0060696300,DKK,Straker Translations Limited,BBB,High,410,BB,Medium,315,BB,Medium,301,STG.CO,1026,BBB,High,348.8
284,DK0010311471,DKK,Sydbank A/S,A,High,500,BB,Medium,300,BB,Medium,301,SYDB.CO,1101,BBB,High,380.3
285,DK0060477503,DKK,Thorney Opportunities Ltd,B,Medium,215,BB,Medium,305,B,Medium,203,TOP.CO,723,B,Medium,238.4


In [84]:
import yfinance as yf
import pandas as pd

# load the original dataframe
df = weighted_scores

# extract unique stock symbols
symbols = df['stock_symbol'].unique()

# create a new dataframe to store the monthly closing data
monthly_close_df = pd.DataFrame()

# loop over each stock symbol and retrieve monthly data using yfinance
for symbol in symbols:
    # retrieve data from yfinance
    stock_data = yf.download(symbol, start='2020-01-01', end='2022-03-01', interval='1mo', progress=False)
    
    # extract the 'Close' column and rename it with the stock symbol
    stock_data = stock_data[['Close']].rename(columns={'Close': symbol})
    
    # add the weighted score for the stock
    weighted_score = df.loc[df['stock_symbol']==symbol, 'weighted_score'].iloc[0]
    stock_data[symbol + '_weighted'] = weighted_score
    
    # append the stock data to the monthly_close_df
    monthly_close_df = pd.concat([monthly_close_df, stock_data], axis=1)

# print the result
print(monthly_close_df)



1 Failed download:
- COLL.ST: Data doesn't exist for startDate = 1577833200, endDate = 1646089200

1 Failed download:
- JOBS.ST: Data doesn't exist for startDate = 1577833200, endDate = 1646089200

1 Failed download:
- MSEIS.OL: No timezone found, symbol may be delisted

1 Failed download:
- SDRL.OL: Data doesn't exist for startDate = 1577833200, endDate = 1646089200
                               AAK.ST  AAK.ST_weighted      ABB.ST  \
Date                                                                 
2019-12-31 23:00:00+00:00  181.050003            454.4  224.800003   
2020-01-31 23:00:00+00:00  165.449997            454.4  208.199997   
2020-02-29 23:00:00+00:00  161.899994            454.4  175.300003   
2020-03-31 22:00:00+00:00  160.449997            454.4  187.500000   
2020-04-30 22:00:00+00:00  171.350006            454.4  185.800003   
2020-05-31 22:00:00+00:00  160.100006            454.4  209.000000   
2020-06-30 22:00:00+00:00  154.800003            454.4  218.500000   

In [85]:
monthly_close_df.dropna()

,AAK.ST,AAK.ST_weighted,ABB.ST,ABB.ST_weighted,ACAD.ST,ACAD.ST_weighted,ADDT-B.ST,ADDT-B.ST_weighted,AEC.ST,AEC.ST_weighted,...,SPNO.CO,SPNO.CO_weighted,STG.CO,STG.CO_weighted,SYDB.CO,SYDB.CO_weighted,TOP.CO,TOP.CO_weighted,TRYG.CO,TRYG.CO_weighted
Date,,,,,,,,,,,,,,,,,,,,,


.to_csv('cleaned_data.csv', index=False)

In [88]:
monthly_close_df.to_csv('new_dataframe1.csv')

In [87]:
stock_data = yf.download(['AEC'], start=start_date, end=end_date, progress=False)

stock_data


1 Failed download:
- AEC: No data found for this date range, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
